In [1]:
#Import Libraries
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import pprint as pp

In [2]:
#Extract Job Title Function
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [3]:
#Extract Job URL Function
def extract_job_url_from_result(soup): 
    urls = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            this_url = a['href']
            to_go_url = "https://www.indeed.com/viewjob" + this_url[7:]
            urls.append(to_go_url)
    return(urls)

In [4]:
#Extract Job Description
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll("span", attrs = {"class": "summary"})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)

In [9]:
#Import the Job Title Dataset
job_titles=pd.read_csv("..\Datasets\Job_Titles.csv", sep = ",")

#Create job_titles list
job_list=list(job_titles.Job_titles)

#Replace spaces with '+'
job_list2=[]
for item in job_list:
    job_list2.append(item.replace(' ', '+'))
    
job_list=job_list2

In [13]:
#Testing the above code
URL ="https://www.indeed.com/jobs?q=data%2420%2C000&l=+New+York&start=10"
    
#conducting a request of the stated URL above:
page = requests.get(URL)

#specifying a desired format of “page” using the html parser
soup = BeautifulSoup(page.text, "html.parser")

pp.pprint(extract_job_title_from_result(soup))

pp.pprint(extract_job_url_from_result(soup))

pp.pprint(extract_summary_from_result(soup))

['Internship: Child Poverty on Violence against Children, Data & Analytic '
 'Section, DRP – NYHQ, Requisition #513831',
 'Data Analyst',
 'Research Associate - Data Collection',
 'Data Entry',
 'HBO Communications Assistant',
 'Junior Data Analyst',
 'CLERK, DATA ENTRY (FULL-TIME) - Summer opportunity',
 'Data Quality Analyst - Computers/Home Theater',
 'SMS/SRC Data Analyst',
 'Intern: Investment Data Scientist']
['https://www.indeed.com/viewjob?jk=258a27b99809a3c4&fccid=b829ee9b3a982489&vjs=3',
 'https://www.indeed.com/viewjob?jk=7de787dd7ed318ab&fccid=667ba717ce627077&vjs=3',
 'https://www.indeed.com/viewjob?jk=9e55238113cf74a8&fccid=5202200066edf11e&vjs=3',
 'https://www.indeed.com/viewjob?jk=5aaec5a7e826bb6d&fccid=88f5eba43de70e76&vjs=3',
 'https://www.indeed.com/viewjob?jk=5db3d9169c925911&fccid=3e3eaa6aedf690a3&vjs=3',
 'https://www.indeed.com/viewjob?jk=4c6f20c4eac41d6b&fccid=e1372f0c6aadd565&vjs=3',
 'https://www.indeed.com/viewjob?jk=30301460376201ec&fccid=c55f4ad42cee2cd3&v